<center><h1> Algorithme KNN</h1></center>

Nous allons découvrir un algorithme d'apprentissage automatique, l'**algorithme des k plus proches voisins** ou **K-NN** (K Nearest Neightbours) : il s'agit d'apporter une réponse plausible, mais pas nécessairement exacte, à un problème de classification auquel il est difficile d'appliquer un algorithme traditionnel.

L'idée est d'utiliser un **grand nombre de données** afin "d'apprendre à la machine" à résoudre un certain type de problème. 

## 1) Exemple visuel

Voici des points, repéré par leurs coordonnées dans un repère ; ces points sont bleus ou rouges. Les coordonnées sont les **descripteurs** des points et la couleur est son **étiquette** (son label).

![](pointsB.png)

On va essayer de **prédire la couleur** de nouveaux points, dont on connaît seulement les coordonnées mais pas l’étiquette.

**Principe :** la couleur du nouveau point sera la couleur majoritaire de ses k
plus proches voisins. 

Ici, on utilise la notion de *distance géométrique* entre deux points, pour déterminer les plus proches voisins.

![](pointsNoir.png)

k =3 : parmi les 3 plus proches voisins l'étiquette majoritaire est rouge ; l'étiquette du point serait rouge.
![](pointsNoir3.png)

k = 5 : parmi les 5 plus proches voisins l'étiquette majoritaire est rouge ; l'étiquette du point serait rouge.

![](pointsNoir5.png)

k = 8 : parmi les 8 plus proches voisins l'étiquette majoritaire est bleue ; l'étiquette du point serait bleue.
![](pointsNoir8.png)

## 2) Le Paramètre k

On remarque que **la décision dépend de la valeur de k**, qui est entier strictement positif.

- lorsque le nombre de voisins est trop petit, cela augmente l'impact d'une perturbation venant d'un point très proche ;

- lorsque le nombre de voisins augmente, cela diminue l'influence de chaque point individuel et les éventuelles perturbations locales ;

- une trop grande valeur de k prends en compte des points qui peuvent être très éloignés et donc peu significatifs.
*Par exemple*, si k est égal au nombre total de points, la prédiction sera toujours la couleur majoritaire du jeu de données.

On va chercher les valeurs de k donnant les **meilleurs prédictions** : elles dépendent à la fois du problème considéré et du jeu de données d'apprentissage.

Pour cela, on coupe le jeu de données en deux jeux : le **jeu d'apprentissage** et le **jeu de test**.

- on fait une prédiction sur un point du jeu de test avec les données du jeu d'apprentissage

- on compare la prédiction avec l'étiquette du point

- en faisant cela avec tous les points du jeu de test, on obtient un taux d'erreurs en fonction du nombre k de voisins choisi, et on peut déterminer la valeur de k qui permet d'avoir le taux d'erreurs le plus faible.

*Remarques :*

   Généralement le choix entre les deux jeux se fait au hasard sur le jeu de données.
   
   On peut également recommencer l'étude en changeant plusieurs foix le jeu d'apprentissage et le jeu de test...

## 3) Notion de distance

Suivant le **problème de classification** étudié, un autre facteur important va être le **choix du calcul de la distance** entre voisins.

Cela va dépendre de la **nature des élements** observés.

- pour la proximité entre deux points du plan, on peut utiliser la *distance euclidienne* :

Avec les coordonnées des points dans un repère orthonormé cela donne : $AB =\sqrt{(x_A-x_B)^2 + (y_A-y_B)^2}$

Mais on peut également utiliser d'autres calculs de distances, comme par exemple la *distance de Manhattan*, où l'on suppose devoir se déplacer en suivant un quadrillage, qui est égale à : $AB = \vert x_A-x_B\vert + \vert y_A-y_B\vert$.

![](distances.png)

- pour la proximité entre deux chaînes de caractères, il y a également plusieurs distances possibles.

par exemple la *distance de Hamming*, qui compte le nombre de caractères (à la même place) par lesquels deux chaînes diffèrent.

    pirouette et girouette sont à une distance de 1 ; marine et copine sont à une distance de 3...

## 4) Implémentation de l'algorithme des KNN

Nous allons utiliser un jeu de données pour faire un exemple. Le jeu de données **"iris"**.

En 1936, Edgar Anderson a collecté des données sur 3 espèces d'iris : **"iris setosa"**, **"iris virginica"** et **"iris versicolor"**.

![](iris.jpg)

Pour chaque iris étudié, Anderson a mesuré (en cm) :
- la largeur des sépales
- la longueur des sépales
- la largeur des pétales
- la longueur des pétales
    
![](descripIris2.png)

Dans le fichier **iris.csv**, vous trouverez  150 de ces mesures.

Maintenant imaginons qu'au cours d'une promenade nous trouvions un iris ; n'étant pas un spécialiste, il ne nous est pas vraiment possible de déterminer la variété. En revanche, nous sommes capables de mesurer la longueur et la largeur des pétales et des cépales de cet iris. 

Nous allons implémenter l'algorithme des k plus proches voisins, afin de déterminer la variété à laquelle appartient notre iris.

In [1]:
import csv                 #module pour travailler avec les fichiers csv
from math import sqrt      #fonction racine carré du module math
from random import shuffle #fonction permettant de mélanger les éléments d'une liste

### 4.1 Importation des données du fichier

Créez une fonction **lecture_donnees()**  qui prend en paramètre le nom de fichier de données csv et retourne les données sous forme d'un **tableau à deux dimensions**.

Le but est d'obtenir **une liste de listes**, où chaque élément représente une ligne du fichier csv. Veillez à supprimer la liste avec le nom des champs du fichier avec la fonction **del()**.

Il vous faudra également transformer les attributs des listes en nombres (excepté le dernier qui correspond à la variété de l'iris), en utilisant la fonction **float()**.

In [2]:
def lecture_donnees(nom_fichier):
    """
    Permet de charger un fichier CSV
    paramètre : nom_fic une chaine de caractères contenant le nom du fichier csv
    résultat : la liste des enregistrements dans le fichier (une liste de listes)
    """
    liste_enreg = []
    with open (nom_fichier, "r", newline="", encoding = "utf-8") as csvfile :
    # création du lecteur csv
        fich_reader = csv.reader(csvfile, delimiter = ",")
        for enreg in fich_reader :
            #enreg est de type list
            #Le premier étant l'entête sous forme d'une liste
            liste_enreg.append(enreg)
    #on supprime le premier élément comportant le nom des champs du fichier
    del(liste_enreg[0])
    #on transforme les 4 premiers éléments de la liste en réels
    for element in liste_enreg:
        for i in range(4):
            element[i] = float(element[i])
    return liste_enreg
iris = lecture_donnees("iris.csv")

In [3]:
assert iris[0][4] == 'Iris-setosa'
assert iris[65][0] == 6.7
assert iris[65][1] == 3.1
assert iris[65][2] == 4.4
assert iris[65][3] == 1.4

### 4.2 Préparation des données

On va séparer l'ensemble de données en deux jeux : le jeu d'apprentissage et le jeu de test. 

Un paramètre p est donné, qui indique le pourcentage de données du jeu à utiliser pour le jeu d'apprentissage, ces données étant choisies aléatoirement ; le reste du jeu de données formera le jeu de test.

Ecrivez la fonction **jeux_donnes()** qui prend en parametres le pourcentage p et le tableau de données. Cette fonction retournera le jeu de données d'apprentissage et le jeu de données de test, sous la forme de **deux listes** comportant les **indices** des données utilisées dans chaque cas. 

*Remarque :* on utilisera la fonction **shuffle()** du module random qui permet de mélanger les indices du jeu de données. Ensuite on prendra la première partie des indices pour le jeu d'apprentissage et le reste pour le jeu de test.

In [4]:
def jeux_donnees(p, donnees):
    """Fonction séparant le jeu de données en deux jeux : un d'entrainement et
    l'autre de test.
    entrées : p réel idiquant le pourcentage de données du jeu à réserver pour
     le jeu d'entrainement donnees qui correspond à la liste du jeu de données
    sorties : un tuple de deux listes comportant les indices des éléments du
     jeu d'entrainement et ceux du jeu de test
    """
    #taille de la liste de donnees
    nombre_donnees = len(donnees)
    #création d'une liste comportant les indices de la liste de données
    liste_indices = [i for i in range(nombre_donnees)]
    #mélange des indices
    shuffle(liste_indices)
    #les p premiers indices sont ceux du jeu d'entrainement
    indices_entrainement = liste_indices[0:p]
    #le reste des indices sont ceux du jeu de test
    indices_test = liste_indices[p:]
    return indices_entrainement, indices_test

#train, test = jeux_donnees(30, iris)

### 4.3 Déterminer le plus proche voisin

Afin de déterminer le plus proche voisin, il est nécessaire de calculer la distance entre deux données.

On va utiliser la **distance euclidienne** : 

la distance entre deux points $x=[x_1,...,x_4]$ et $y=[y_1,...,y_4]$ ayant quatre coordonnées, est

$$d(x,y) = \sqrt{(x_1-y_1)^2 + (x_2-y_2)^2 + (x_3-y_3)^2 + (x_4-y_4)^2}$$

*Remarque :* Il s'agit d'une généralisation de la définition de distance entre deux points du plan ; ici on a quatres nombres (les descripteurs) qui définissent chaque donnée : la largeur et la longueur des pétales et des sépales.

Ecrivez la fonction **distance(x,y)**, qui prend les coordonnées de deux points en paramètres et qui retourne la distance euclidienne entre ces deux points.

In [5]:
def distance(x,y):
    """Fonction qui calcule la distance euclidienne entre deux points.
    entrée : deux listes de réels, de mêmes tailles
    sortie : un réel égal à la distance euclidienne entre x et y
    """
    somme = 0
    for i in range(len(x)):
        #on fait la somme du carré des écarts
        somme = somme + (x[i]-y[i])**2
    #on prend la racine carré du résultat
    return sqrt(somme)

#on vérifie la distance entre la première iris et la 46 ième ;
#on prend les 4 premiers éléments de chaque iris, ses descripteurs.
assert distance(iris[0][0:4], iris[45][0:4]) == 0.5916079783099616

En utilisant la fonction distance précédente, écrivez la fonction **lePlusProcheVoisin(x,entrainement,donnees)** qui prend en paramètres les mesures d'une iris **x**, la liste des indices du jeu d'apprentissage et le tableau de données. 

Cette fonction retournera **l'indice du jeu d'apprentissage** correspondant à l'élément le plus proche de **x**.

In [6]:
def lePlusProcheVoisin(x,entrainement,donnees):
    """Fonction donnant l'indice de l'élément le plus proche de x parmi les iris
    du jeu d'entrainement.
    entrées : x une liste de quatre réels correspondant aux mesures d'une iris ;
       entrainement correspondant à la liste des indices du jeu d'entrainement ;
       donnees la liste du jeu de données iris.
    sortie : l'indice de l'élément du jeu d'entrainement le plus proche de x
    """
    #initialisation avec le premier indice de la liste entrainement
    indice_du_plus_proche = entrainement[0]
    #initialisation avec le premier élément du jeu d'entrainement
    distance_min = distance(x, donnees[indice_du_plus_proche][0:4])
    #on calcule la distance avec chaque élément du jeu d'entrainement
    for indice in entrainement:
        #si celle-ci est plus petite on change l'indice et la distance minimale
        if distance(x, donnees[indice][0:4]) < distance_min:
            indice_du_plus_proche = indice
            distance_min = distance(x, donnees[indice][0:4])
    return indice_du_plus_proche

#on cherche le plus proche voisin de x =[5, 3, 3, 2] sur tout le jeu de données iris
assert lePlusProcheVoisin([5, 3, 3, 2],[i for i in range(150)],iris) == 98

### 4.4 Déterminer les k plus proches voisins

Ecrivez la fonction **lesKplusProchesVoisins()** qui prend pour paramètres **x** les mesures de notre iris, **k** le nombre de voisins souhaités, **donnees** qui comprend le jeu de données iris et **entrainement** la liste des indices du jeu d'apprentissage.

Cette fonction retournera la **liste des indices** des k voisins les plus proches de **x** dans le jeu d'apprentissage.


In [7]:
def lesKplusProchesVoisins(x, k , donnees, entrainement):
    """Fonction qui permet de déterminer les k plus proches voisins de x dans
     le jeu d'apprentissage.
     Entrées : x une liste comportant les 4 descripteurs de l'iris ; k le nombre
     entier positif désignant le nombre de voisins souhaités ; donnees le jeu de
     données iris sous forme d'une liste de listes et entrainement la liste des
     indices des éléments du jeu d'entrainement.
     sortie : la liste des indices des k plus proches voisins
    """
    k_indices =[]
    for i in range(k):
        n = lePlusProcheVoisin(x, entrainement, donnees)
        #on ajoute l'indice de plus proche voisin
        k_indices.append(n)
        #on garde tous les éléments de la liste entrainement, sauf n
        entrainement = [indice for indice in entrainement if indice != n]
    return k_indices

#on cherche les 3 plus proches voisins de x =[5, 3, 3, 2] sur tout le jeu de données iris
assert lesKplusProchesVoisins([5, 3, 3, 2], 3 , iris, [i for i in range(150)]) == [98, 64, 59]

### 4.5 Prédiction de la variété d'une iris

On peut obtenir les k plus proches voisins de notre iris, il reste à regarder qu'elle est la variété majoritaire parmi ses voisins ; ce sera notre **prédiction** sur celle de notre iris ! 

Ecrivez la fonction **prediction(listeVoisins, donnees)** qui prend en paramètre les données **iris** et la liste des indices des voisins **listeVoisins**. Cette fonction retournera la variété majoritaire des voisins.

Vous pouvez utiliser un dictionnaire dont les clés seront les différentes variétés d'iris, et la valeur le nombre d'iris de cette variété dans la liste des voisins.

In [8]:
def prediction(listeVoisins, donnees):
    """Fonction permettant d'obtenir la variété majoritaire dans la liste
     des voisins.
     entrées : listeVoisins est la liste des indices des plus proches voisins et
           donnees est le jeu de données Iris, sous forme d'une liste de listes.
     sortie : une chaine de carctère donnant la variété majoritaire.
    """
    #création du dictionnaire avec les différentes variétés possibles
    varietes = {"Iris-setosa" : 0, "Iris-versicolor" : 0, "Iris-virginica" : 0}
    #on parcour la liste des voisins en regardant la variété de chacun
    for indice in listeVoisins:
        #on augmente la valeur de la variété dans le dictionnaire
        varietes[donnees[indice][4]] +=1
    #on cherche la valeur maximale dans le dico
    max = 0
    cle_max = ""
    for cle in varietes:
        if varietes[cle] > max:
            max = varietes[cle]
            cle_max = cle
    return cle_max

#Par exemple, avec la liste des voisins [98, 64, 5], la variété majoritaire sera `Iris-versicolor`.
assert prediction([98, 64, 5], iris) == 'Iris-versicolor'

Comment **déterminer la variété probable d'un iris** ayant pour mesures [6,3,3,4], en utilisant 5 voisins et 60 %  des données pour votre jeu d'apprentrissage, à l'aide des fonctions crées aux questions précédentes ?
Testez votre méthode plusieurs fois de suites ; obtenez-vous toujours la même réponse ?

In [9]:
jeu_apprentissage, jeu_test = jeux_donnees(60, iris)
print(prediction(lesKplusProchesVoisins([6,3,3,4], 5, iris, jeu_apprentissage), iris))

Iris-versicolor


### 4.6 Utilisation du jeu de test et taux d'erreur sur les prédictions 

On fixe p le pourcentage de données réservé au jeu d'apprentissage à 50%. En utilisant la fonction **jeux_donnees()**, on défini les listes d'indices **jeu_appprentissage** et **jeu_test**.

In [10]:
#on prépare les jeux d'apprentissage et de test
jeu_apprentissage, jeu_test = jeux_donnees(50, iris)

Ecrivez une fonction **taux_erreurs()** qui prendra en paramètres le nombre **k** de voisins proches, la liste **train** des indices du jeu d'apprentissage et celle **test** du jeu de test, ainsi que **donnees** le jeu de données iris ; elle
retournera le **taux d'erreur sur le jeu de test** de l'algorithme des k plus proches voisins.

C'est à dire le taux de prédictions inexactes faites sur les données du jeu de test.

In [11]:
def taux_erreurs(k, train, test, donnees):
    """Fonction qui retourne le pourcentage d'erreurs de prédictions faites sur
    la variété des iris du jeu de test.
    entrées : k est le nombre de voisins proches à examiner ; train est la liste
     des indices du jeu d'apprentissage ; test la liste des indices du jeu de
     test ; données est le jeu de données iris sous forme d'une liste de listes.
    sortie : un réel exprimant le porcentage d'erreurs commises par l'algorithme KNN
    """
    #on examine chaque iris du jeu de test
    nbre_erreurs = 0          #nombre d'erreurs commises
    for element in test:
        #on fait une prédiction
        predic = prediction(lesKplusProchesVoisins(donnees[element][0:4], k, donnees, train), donnees)
        #on la compare à la variété réelle
        if predic != donnees[element][4]:
            nbre_erreurs += 1
    return nbre_erreurs/(len(test))*100     #on calcule le pourcentage d'erreurs

print(taux_erreurs(3,jeu_apprentissage,jeu_test,iris))


3.0


### 4.7 Détermination du choix du nombre de voisins

Le paramètre $k$ est un facteur important pour la qualité de l'algorithme. 

Pour chaque nombre de voisins **k** possible, on va déterminer le taux d'erreurs sur le jeu test.

Ecrivez la fonction **meilleurK()** prenant en paramètres la liste **train** des indices du jeu d'apprentissage et celle **test** du jeu de test, ainsi que **donnees** le jeu de données iris. Elle retournera la valeur **k** donnant le moins d'erreurs sur le jeu de test.

In [12]:
def meilleurK(train, test, donnees):
    """Fonction qui permet de déterminer le nombre k de voisins permettant de faire
    le moins d'erreurs possibles lors des prédictions.
    entrées : train est la liste des indices du jeu d'apprentissage ; test la
     liste des indices du jeu de test ; données est le jeu de données iris
      sous forme d'une liste de listes.
    sortie : un entier positif représentant le nombre de voisins
    """
    #initialisation des variables
    k_optimal = 0
    taux_min = 100
    #on essaye toutes les valeurs possibles de k
    for k in range(len(train)):
        #on regarde si le taux d'erreur est plus petit
        if taux_erreurs(k, train, test, donnees) < taux_min:
            taux_min = taux_erreurs(k, train, test, donnees)
            k_optimal = k
    return k_optimal

meilleurK(jeu_apprentissage, jeu_test, iris)

2

## 5) Conclusion

   Pourquoi l'**apprentissage automatique** ou **machine learning** est tant "à la mode" depuis quelques années ? 
   
   Simplement parce que le nerf de la guerre dans les algorithmes de machine learning est la **qualité** et la **quantité des données** (les données qui permettront à la machine d'apprendre à résoudre un problème), or, avec le développement d'internet, il est relativement simple de trouver des données sur n'importe quel sujet (on parle de **"big data"**). 
   
   À noter aussi l'importance des stratégies mises en place par les **GAFAM** (Google, Apple, Facebook, Amazon et Microsoft) afin de récupérer un grand nombre de données concernant leurs clients. Ces données sont très souvent utilisées pour "nourrir" des algorithmes de machine learning (comment, d'après vous, Amazon arrive à proposer à ces clients des "suggestions d'achats" souvent très pertinentes ?)...

-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------

*Référence :* TP KNN du DIU EIL (2019-2020) à Orleans par madame **Thi-Bich-Hanh Dao**.